In [2]:
import numpy as np
import sys
sys.path.append("..")

from Code.VQLSSVM import VQLSSVM
from Code.Utils import prepareDataset
import random
import time

np.set_printoptions(precision=10, suppress=True)

In [1]:
averageOf = 10
shots = 10000
qubits = 3
gamma: float = 0.01 # regularization parameter
subsetSize: int = 2**qubits - 1 # number of training points
classToFilterOut: int = 2
quantumSimulation: bool = False

random.seed(0)
xTrain, xTest, yTrain, yTest = prepareDataset(normalizeValues=True, dataset="iris",subsetSize=subsetSize, classToFilterOut=classToFilterOut)

vqlssvmVectors: VQLSSVM = VQLSSVM(gamma, shots)

print("non parallel")
totalTime = 0
random.seed(0)
for i in range(averageOf):
    start = time.time()
    vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=quantumSimulation, verbose=False, iterations = 200, method="COBYLA")
    end = time.time()
    totalTime += end - start

print("Time taken: ", totalTime/averageOf)


print("worker sizes")

bestTime = time.time()
bestWorkers = 0
bestJobSize = 0

print("parallel")
coreList = [2,4,8,10]
jobList = [1,2,4,8,10]
for i in coreList:
    for j in jobList:
        totalTime = 0
        random.seed(0)
        for _ in range(averageOf):
            start = time.time()
            vqlssvmVectors.train(xTrain, yTrain, quantumSimulation=quantumSimulation, verbose=False, iterations = 200, method="COBYLA", threads=i, jobs=j, circuitConstructionThreading = True)
            end = time.time()
            totalTime += end - start
        timeCirc = totalTime/averageOf

        print("workers: ", i, "jobSize: ", j, "time: ", timeCirc)
        if timeCirc < bestTime:
            bestTime = timeCirc
            bestWorkers = i
            bestJobSize = j

print("best time: ", bestTime)
print ("best workers: ", bestWorkers)
print ("best job size: ", bestJobSize)

ModuleNotFoundError: No module named 'Code'